**IBM Applied Data Science Capstone Course by Coursera**

 * *Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.*
 * *Get the geographical coordinates of the neighborhoods in Toronto.*
 * *Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).*
 
 -------
 **1. Import Library**


In [ ]:
import numpy as np #to handle file in vectorize manner
import pandas as pd #for data analysis
pd.set_option('display.max_columns',none)
pd.se